In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import random
from pprint import pprint
# from helper_functions import determine_type_of_feature

In [3]:
df = pd.read_csv("cleaned_income2.csv")

In [4]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [5]:
#Spliting the dataset into train and test data into (80% : 20%) respectively
random.seed(0)
train_df, test_df = train_test_split(df, test_size=.2)

# Display the shapes of the resulting sets
print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)

# Save the training and testing sets to separate CSV files
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)
train_df[['income']].to_csv('train_df_labels.csv', index=False)
test_df[['income']].to_csv('test_df_labels.csv', index=False)

Training set shape: (39074, 15)
Testing set shape: (9768, 15)


## Helper Functions

In [6]:
data = train_df.values
data[:5]

array([[25, 'Private', 226802, '11th', 7, 'Never-married',
        'Machine-op-inspct', 'Own-child', 'Black', 'Male', 0, 0, 40,
        'United-States', '<=50K'],
       [28, 'Local-gov', 336951, 'Assoc-acdm', 12, 'Married-civ-spouse',
        'Protective-serv', 'Husband', 'White', 'Male', 0, 0, 40,
        'United-States', '>50K'],
       [44, 'Private', 160323, 'Some-college', 10, 'Married-civ-spouse',
        'Machine-op-inspct', 'Husband', 'Black', 'Male', 7688, 0, 40,
        'United-States', '>50K'],
       [34, 'Private', 198693, '10th', 6, 'Never-married',
        'Other-service', 'Not-in-family', 'White', 'Male', 0, 0, 30,
        'United-States', '<=50K'],
       [29, 'Private', 227026, 'HS-grad', 9, 'Never-married',
        'Prof-specialty', 'Unmarried', 'Black', 'Male', 0, 0, 40,
        'United-States', '<=50K']], dtype=object)

## Data Pure or Not?

In [7]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

## Classify

In [8]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

## Potential Splits

In [9]:
# def get_potential_splits(data):
    
#     potential_splits = {}
#     _, n_columns = data.shape
#     for column_index in range(n_columns - 1):          # excluding the last column which is the label
#         values = data[:, column_index]
#         unique_values = np.unique(values)
        
#         type_of_feature = FEATURE_TYPES[column_index]
#         if type_of_feature == "continuous":
#             potential_splits[column_index] = []
#             for index in range(len(unique_values)):
#                 if index != 0:
#                     current_value = unique_values[index]
#                     previous_value = unique_values[index - 1]
#                     potential_split = (current_value + previous_value) / 2

#                     potential_splits[column_index].append(potential_split)
        
#         # feature is categorical
#         # (there need to be at least 2 unique values, otherwise in the
#         # split_data function data_below would contain all data points
#         # and data_above would be empty)
#         elif len(unique_values) > 1:
#             potential_splits[column_index] = unique_values
    
#     return potential_splits

def get_potential_splits(data, random_subspace):
    
    potential_splits = {}
    _, n_columns = data.shape
    column_indices = list(range(n_columns - 1))    # excluding the last column which is the label
    
    if random_subspace and random_subspace <= len(column_indices):
        column_indices = random.sample(population=column_indices, k=random_subspace)
    
    for column_index in column_indices:          
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

## Lowest Overall Entropy

In [10]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [11]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [12]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

## Split Data

In [13]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

## Decision Tree

### Representation of the Decision Tree

In [14]:
sub_tree = {"question": ["yes_answer", 
                         "no_answer"]}

In [15]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,Private,103497,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,30,United-States,<=50K


## Determine Type of Feature

In [16]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "income":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

## Algorithm

In [17]:
def d_t_algorithm(df, counter=0, min_samples=2, max_depth=5, random_subspace=None):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data, random_subspace)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = d_t_algorithm(data_below, counter, min_samples, max_depth, random_subspace)
        no_answer = d_t_algorithm(data_above, counter, min_samples, max_depth, random_subspace)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

## Make Predictions

In [18]:
# One example Prediction
def predict_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

In [19]:
def d_t_predictions(test_df, tree):
    predictions = test_df.apply(predict_example, args=(tree,), axis=1)
    return predictions

## Random Forest

In [20]:
def bootstrapping(train_df, n_bootstrap):
    bootstrap_indices = np.random.randint(low=0, high=len(train_df), size=n_bootstrap)
    df_bootstrapped = train_df.iloc[bootstrap_indices]
    
    return df_bootstrapped

def r_f_algorithm(train_df, n_trees, n_bootstrap, n_features, dt_max_depth):
    forest = []
    for i in range(n_trees):
        df_bootstrapped = bootstrapping(train_df, n_bootstrap)
        tree = d_t_algorithm(df_bootstrapped, max_depth=dt_max_depth, random_subspace=n_features)
        forest.append(tree)
    
    return forest

def r_f_predictions(test_df, forest):
    df_predictions = {}
    for i in range(len(forest)):
        column_name = "tree_{}".format(i)
        predictions = d_t_predictions(test_df, tree=forest[i])
        df_predictions[column_name] = predictions

    df_predictions = pd.DataFrame(df_predictions)
    r_f_predictions = df_predictions.mode(axis=1)[0]
    
    return r_f_predictions

## Classification

In [21]:
sub_tree

{'question': ['yes_answer', 'no_answer']}

In [22]:
example = test_df.iloc[0]
example

age                                43
workclass                     Private
fnlwgt                          99212
education                   Assoc-voc
educational-num                    11
marital-status     Married-civ-spouse
occupation            Exec-managerial
relationship                  Husband
race                            White
gender                           Male
capital-gain                     3103
capital-loss                        0
hours-per-week                     48
native-country          United-States
income                           >50K
Name: 25247, dtype: object

## Calculate Accuracy

In [23]:
def calc_accuracy(predictions, labels):
    predictions_correct = predictions == labels
    accuracy = predictions_correct.mean()
    
    return accuracy

In [29]:
forest = r_f_algorithm(train_df, n_trees=20, n_bootstrap=900, n_features=2, dt_max_depth=20)
predictions = r_f_predictions(test_df, forest)
accuracy = calc_accuracy(predictions, test_df.income)*100

print("Accuracy = {}".format(accuracy))

Accuracy = 85.16584766584766
